In [ ]:
import os
import asyncio
from datetime import datetime
from pydantic import BaseModel, PrivateAttr
from mirascope.core import BaseMessageParam, google, prompt_template
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
maps_key = os.environ.get("GOOGLE_MAPS_API_KEY")


In [3]:

import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key=maps_key)

# Geocoding an address
geocode_result = gmaps.geocode('1600 Amphitheatre Parkway, Mountain View, CA')
print(geocode_result)

# Look up an address with reverse geocoding
reverse_geocode_result = gmaps.reverse_geocode((40.714224, -73.961452))
print(reverse_geocode_result)

# Request directions via public transit
now = datetime.now()
directions_result = gmaps.directions("Sydney Town Hall",
                                     "Parramatta, NSW",
                                     mode="transit",
                                     departure_time=now)
print(directions_result)

# Validate an address with address validation
addressvalidation_result =  gmaps.addressvalidation(['1600 Amphitheatre Pk'], 
                                                    regionCode='US',
                                                    locality='Mountain View', 
                                                    enableUspsCass=True)
print(addressvalidation_result)
# # Get an Address Descriptor of a location in the reverse geocoding response
# address_descriptor_result = gmaps.reverse_geocode((40.714224, -73.961452), enable_address_descriptor=True)


INFO:googlemaps.client:API queries_quota: 60


[{'address_components': [{'long_name': '1600', 'short_name': '1600', 'types': ['street_number']}, {'long_name': 'Amphitheatre Parkway', 'short_name': 'Amphitheatre Pkwy', 'types': ['route']}, {'long_name': 'Mountain View', 'short_name': 'Mountain View', 'types': ['locality', 'political']}, {'long_name': 'Santa Clara County', 'short_name': 'Santa Clara County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'California', 'short_name': 'CA', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}, {'long_name': '94043', 'short_name': '94043', 'types': ['postal_code']}, {'long_name': '1351', 'short_name': '1351', 'types': ['postal_code_suffix']}], 'formatted_address': '1600 Amphitheatre Pkwy, Mountain View, CA 94043, USA', 'geometry': {'location': {'lat': 37.4220108, 'lng': -122.084748}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': 37.4233655802915, 'lng': -122.082

In [4]:
class LocalizedRecommenderBase(BaseModel):
    history: list[BaseMessageParam | google.GoogleMessageParam] = []
    _gmaps: googlemaps.Client = PrivateAttr()

    def __init__(self, **data):
        super().__init__(**data)
        # Initialize Google Maps client with API key from environment variable
        self._gmaps = googlemaps.Client(key=maps_key)

    async def google_maps_places(self, place_id: str):
        """
        Get details of a place on Google Maps using Places API.
        """
        logger.info(f"Calling google_maps_places with place_id: {place_id}")
        # The Google Maps client is synchronous, so run in executor for async compatibility
        loop = asyncio.get_running_loop()
        place_details = await loop.run_in_executor(
            None, lambda: self._gmaps.place(place_id=place_id)
        )
        result = place_details.get("result", {})
        logger.info(f"google_maps_places result: {result}")
        return {
            "opening_hours": result.get("opening_hours", {}).get("weekday_text", []),
            "rating": result.get("rating", ""),
            "name": result.get("name", ""),
        }

    async def _google_maps_search(self, latitude: float, longitude: float, query: str):
        """
        Search for places near given coordinates using Places API.
        """
        logger.info(f"Calling _google_maps_search with latitude: {latitude}, longitude: {longitude}, query: {query}")
        loop = asyncio.get_event_loop()
        places_result = await loop.run_in_executor(
            None,
            lambda: self._gmaps.places_nearby(
                location=(latitude, longitude),
                keyword=query,
                radius=5000  # radius in meters, adjust as needed
            ),
        )
        search_results = places_result.get("results", [])
        results = []
        for place in search_results:
            place_id = place.get("place_id")
            if place_id:
                details = await self.google_maps_places(place_id)
                results.append(details)
        logger.info(f"_google_maps_search results: {results}")
        return results

    async def _get_current_date(self):
        """Get the current date and time."""
        logger.info("Calling _get_current_date")
        loop = asyncio.get_event_loop()
        current_date = await loop.run_in_executor(
            None, lambda: datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        )
        logger.info(f"_get_current_date result: {current_date}")
        return current_date

    async def _get_coordinates_from_location(self, location_name: str):
        """
        Get the coordinates of a location using Geocoding API.
        """
        logger.info(f"Calling _get_coordinates_from_location with location_name: {location_name}")
        loop = asyncio.get_event_loop()
        geocode_result = await loop.run_in_executor(
            None, lambda: self._gmaps.geocode(location_name)
        )
        if geocode_result:
            location = geocode_result[0]["geometry"]["location"]
            latitude = location["lat"]
            longitude = location["lng"]
            logger.info(f"_get_coordinates_from_location result: Latitude: {latitude}, Longitude: {longitude}")
            return f"Latitude: {latitude}, Longitude: {longitude}"
        else:
            logger.warning("_get_coordinates_from_location: No location found")
            return "No location found, ask me about a specific location."

In [ ]:
class LocalizedRecommenderBaseWithStep(LocalizedRecommenderBase):
    @google.call(model="gemini-2.0-flash", stream=True)
    @prompt_template(
        """
        SYSTEM:
        You are a local guide that recommends the best places to visit in a location.
        Use the `_get_current_date` function to get the current date.
        Use the `_get_coordinates_from_location` function to get the coordinates of a location if you need it.
        Use the `_google_maps_search` function to get the best places to visit in a location based on the users query.

        MESSAGES: {self.history}
        USER: {question}
        """
    )
    async def _step(self, question: str) -> google.GoogleDynamicConfig:
        return {
            "tools": [
                self._get_current_date,
                self._get_coordinates_from_location,
                self._google_maps_search,
            ]
        }

In [ ]:
class LocalizedRecommender(LocalizedRecommenderBaseWithStep):
    async def _get_response(self, question: str):
        if not question.strip():
            logger.error("Empty question passed to _get_response.")
            return
        
        logger.info(f"Starting _get_response with question: {question}")
        response = await self._step(question)
        logger.info("Received response from _step")

        tool_call = None
        output = None

        async for chunk, tool in response:
            if tool:
                logger.info(f"Tool detected: {tool}")
                output = await tool.call()
                logger.info(f"Tool output: {output}")
                tool_call = tool
            else:
                if chunk.content:
                    print(chunk.content, end="", flush=True)

        if response.user_message_param:
            self.history.append(response.user_message_param)

        self.history.append(response.message_param)

        if tool_call and output:
            logger.info("Appending tool call and output to history")
            self.history += response.tool_message_params([(tool_call, str(output))])
            return await self._get_response(question)
        
        logger.info("Completed _get_response")
        return

    async def run(self):
        logger.info("Starting run loop")
        while True:
            question = input("(User): ").strip()
            if question.lower() == "exit":
                logger.info("Exiting run loop")
                break

            if not question:
                print("(Assistant): It's quiet in here. Please provide a valid input.")
                logger.warning("Empty input detected. Prompting user again.")
                continue

            logger.info(f"Received user question: {question}")
            print("(Assistant): ", end="", flush=True)
            await self._get_response(question)
            print()
        logger.info("Run loop ended")

In [7]:
recommender = LocalizedRecommender(history=[])
await recommender.run()

INFO:googlemaps.client:API queries_quota: 60
INFO:__main__:Starting run loop


(Assistant): It's quiet in here. Please provide a valid input.


INFO:__main__:Received user question: Where can I get good vegan pizza in the Cape Town city bowl?


(Assistant): 

INFO:__main__:Starting _get_response with question: Where can I get good vegan pizza in the Cape Town city bowl?
INFO:__main__:Received response from _step
INFO:google_genai.models:AFC is enabled with max remote calls: 10.


...

INFO:__main__:Received user question: Where can I get good vegan pizza in the Cape Town city center?


(Assistant): 

INFO:__main__:Starting _get_response with question: Where can I get good vegan pizza in the Cape Town city center?
INFO:__main__:Received response from _step
INFO:google_genai.models:AFC is enabled with max remote calls: 10.


INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:streamGenerateContent?alt=sse "HTTP/1.1 400 Bad Request"


ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Unable to submit request because it has an empty text parameter. Add a value to the parameter and try again. Learn more: https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini', 'status': 'INVALID_ARGUMENT'}}